In [169]:
from pypdf import PdfReader
import re

# Extraction et traitement des données


## On extrait le texte du pdf

In [170]:
reader = PdfReader('Star_Wars_Episode_3_Revenge_Of_The_Sith.pdf')
npages = len(reader.pages)
text = ''
for i in range(npages):
    page = reader.pages[i]
    text += page.extract_text()


## On traite le texte pour diviser le script en scènes

In [171]:
temp = text.split('\n')
data = []

# On localise les scènes
scenes = []
for i in range(len(temp)):
    # George Lucas a oublié de mettre des points à tous les INT et EXT...
    if ("INT" in temp[i] or "EXT" in temp[i]) and "INTERCUT" not in temp[i]:
        scenes.append(i)

# On divise les scènes dans data
for i in range(len(scenes)-1):
    data.append(temp[scenes[i]:scenes[i+1]])
data.append(temp[scenes[-1]:])




## On cherche les personnages parlant dans chaque scène

In [172]:
characters = {}
pattern = r'\b[A-Za-z][A-Za-z0-9\- ]+:'
for i in range(len(data)):
    temp = ''.join(data[i][1:])
    personnages = re.findall(pattern, temp)
    personnages = [personnages[m].upper() for m in range(len(personnages))]
    if i not in characters:
        characters[i] = []
    for personnage in personnages:
        if personnage[:-1] not in characters[i]:
            characters[i].append(personnage[:-1])


## On corrige les fautes de frappe

In [173]:
error = 0
# G-3PO
characters[161].remove('G-3PO')
error += 1
 
# Qui -Gon
characters[221].remove('QUI -GON')
error += 1

# ANAKINN
characters[52].remove('ANAKINN')
error += 1

# Obi-Wan  (avec espace)
characters[85].remove('OBI-WAN ')
error += 1

# Dabth sidious
characters[140].remove('DABTH SIDIOUS')
characters[140].append('DARTH SIDIOUS')
error += 1

# Some errors
characters[7].remove(characters[7][3])
characters[129].remove(characters[129][0])
characters[115].remove(characters[115][0])
characters[104].remove(characters[104][2])
error += 4

for i in characters:
    if 'MACE' in characters[i]:
        characters[i].remove('MACE')
        characters[i].append('MACE WINDU')
        error += 1

    if 'MACE WLNDU' in characters[i]:
        characters[i].remove('MACE WLNDU')
        characters[i].append('MACE WINDU')
        error += 1

    if 'BAIL' in characters[i]:
        characters[i].remove('BAIL')
        characters[i].append('BAIL ORGANA')
        error += 1

    if 'DARTH SLDIOUS' in characters[i]:
        characters[i].remove('DARTH SLDIOUS')
        characters[i].append('DARTH SIDIOUS')
        error += 1

print("Nombre d'erreurs (hors fautes de majuscule): ", error)

Nombre d'erreurs (hors fautes de majuscule):  25


# Début du graphe

## On compte le nombre de noeuds uniques

In [ ]:
nodes = []
edges = []
for i in characters:

    if len(characters[i]) >= 2:
        edges.append(characters[i])

    for j in characters[i]:
        if j not in nodes:
            nodes.append(j)
            
# sort nodes alphabetically
nodes.sort()
Nnode = len(nodes)

# Faire la liste d'adjacence
adj = {}
for node in nodes:
    adj[node] = []

# Remplir la liste d'adjacence
for edge in edges:
    for character in edge:
        
        



ANAKIN
OBI-WAN
 
OBI-WAN
ANAKIN
 
OBI-WAN
ANAKIN
ODD BALL
 
ANAKIN
OBI-WAN
ODD BALL
CLONE PILOT 2
 
OBI-WAN
ANAKIN
 
GENERAL GRIEVOUS
CAPTAIN
 
BODYGUARD I
OBI-WAN
ANAKIN
 
OBI-WAN
ANAKIN
 
OBI-WAN
ANAKIN
 
ANAKIN
BATTLE DROID
 
OBI-WAN
ANAKIN
 
SUPER BATTLE DROID 1
SUPER BATTLE DROID 2
 
DROID 1
DROID 2
 
SUPER BATTLE DROID 1
OBI-WAN
 
ANAKIN
OBI-WAN
 
OBI-WAN
ANAKIN
PALPATINE
COUNT DOOKU
 
PALPATINE
ANAKIN
 
GENERAL GRIEVOUS
PILOT
 
GENERAL GRIEVOUS
PILOT
 
ANAKIN
OBI-WAN
 
OBI-WAN
ANAKIN
 
OBI-WAN
ANAKIN
PALPATINE
 
BODYGUARD
GENERAL GRIEVOUS
 
ANAKIN
OBI-WAN
PALPATINE
SUPER BATTLE DROID
 
GENERAL GRIEVOUS
OBI-WAN
ANAKIN
PILOT
 
ANAKIN
OBI-WAN
FLRESHIP PILOT
 
ANAKIN
OBI-WAN
PALPATINE
BAIL ORGANA
C-3PO
MACE WINDU
 
BAIL ORGANA
ANAKIN
PADME
 
SUPER BATTLE DROID G21
GENERAL GRIEVOUS
 
GENERAL GRIEVOUS
DARTH SIDIOUS
 
ANAKIN
PADME
 
PADME
ANAKIN
 
YODA
ANAKIN
 
OBI-WAN
ANAKIN
 
PALPATINE
ANAKIN
 
BAIL ORGANA
FANG ZAR
PADME
MON MOTHMA
GIDDEAN DANU
TERR TANEEL
GLDDEAN DANU
 
ANAKIN
YODA
